In [1]:
import json
import pdfplumber
import spacy
import pandas as pd
import numpy as np
import os
import unidecode
import PyPDF2
from tika import parser
from spacy.matcher import PhraseMatcher
from spacy import displacy
from progress_bar.progress_bar import printProgressBar
import torch
import gc
import multiprocessing
import sys
from flair.data import Sentence
from flair.models import SequenceTagger

        

In [2]:
tagger = SequenceTagger.load("flair/ner-english-large")

2022-08-03 00:17:04,309 loading file /home/heider/.flair/models/ner-english-large/07301f59bb8cb113803be316267f06ddf9243cdbba92a4c8067ef92442d2c574.554244d3476d97501a766a98078421817b14654496b86f2f7bd139dc502a4f29
2022-08-03 00:17:40,138 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


In [3]:
nlp_spa = spacy.load('en_core_web_trf')

In [4]:
#spacy.prefer_gpu()

In [5]:
nlp_compare = spacy.load('en_core_web_lg')
nlp_person = spacy.load('en_core_web_trf')
#nlp_spa = spacy.load('en_core_web_sm')
#nlp_person = spacy.load('es_core_news_lg')
#nlp_spa = spacy.load('en_core_web_sm')
#nlp_person = spacy.load('es_core_news_lg')
#nlp_compare = spacy.load('en_core_web_lg')
#nlp_person = spacy.load('en_core_web_trf')
#nlp_person = spacy.load('es_dep_news_trf')

In [6]:
def clean_break_line(docs):
    new_docs = docs.replace('\n',"")
    return new_docs
    
def clean_spaces_text(text):
    new_text = " ".join(text.split())
    return(new_text)
    
def clean_points_text(docs):
    new_docs = docs.replace('.',"")
    return new_docs

def clean_commas_text(docs):
    new_docs = docs.replace(',',"")
    return new_docs

def clean_apostrophe(docs):
    new_docs = docs.replace("'","")
    return new_docs

def clean_accents(text):
    unaccented_string = unidecode.unidecode(text)
    return unaccented_string
    #lower_u_string = unaccented_string.lower()
    #return lower_u_string

def split_authors(text):
    return text.split(";")

def contains_text_list(names, authors, context):
    df_names = []
    df_authors = []
    df_text = []
    df_lower_authors = []
    df_context = []
    for aut in authors:
        for elem, cont in zip(names, context):
            df_names.append(elem)
            df_lower_authors.append(clean_accents(aut.lower()))
            df_text.append(contains_text(clean_accents(elem.lower()), clean_accents(aut.lower())))
            df_authors.append(aut)
            df_context.append(cont)
    df = {'names': df_names,'author': df_authors, 'author_lower': df_lower_authors,'contains': df_text,
          'context': df_context}
    df = pd.DataFrame(df)
    #print(df)
    return df

def name_in_title(data_frame, title):
    can_tutor = data_frame.copy()
    #print(title)
    can_tutor['is_in_title'] = can_tutor['tutor_lower'].apply(contains_text, args = title)
    #print(can_tutor)
    
    false_values = can_tutor[can_tutor['is_in_title']==False]
    return false_values
   

def contains_text(text1, text2):
    text_1 = text1.split(" ").copy()
    text_1 = list(map(clean_accents, text_1))
    text_1 = list(map(clean_apostrophe, text_1))
    text_1 = list(map(clean_commas_text, text_1))
    text_1 = list(map(clean_points_text, text_1))
    text_2 = text2.split(" ").copy()
    text_2 = list(map(clean_accents, text_2))
    text_2 = list(map(clean_apostrophe, text_2))
    text_2 = list(map(clean_commas_text, text_2))
    text_2 = list(map(clean_points_text, text_2))
    #print(text_1)
    #print(text_2)
    for word_1 in text_1:
        if word_1 != None and len(word_1)>2:
            for word_2 in text_2:
                if word_2!= None and word_1.lower() == word_2.lower():
                    #print(word_1, "<->", word_2)
                    return True
            
    for word_2 in text_2:
        if word_2 != None and len(word_2)>2:
            for word_1 in text_1:
                if word_1!= None and word_2.lower() == word_1.lower():
                    #print(word_2, "<->", word_1)
                    return True
    return False

def pick_candidate_tutor(data_frame, title):
    check_multiple = data_frame.copy()
    
    values = ["Dios", "JOSE GREGORIO HERNANDEZ", "Brs"]
    values = map(str.lower, values)
    
    #here we delete dios and JOSE GREGORIO HERNANDEZ and return the first name.
    true_values = check_multiple[check_multiple["contains"]==True]
    t_names = true_values["names"]
    t_names = t_names.to_list()
    #print(check_multiple)
    #print(t_names)
    tutor = check_multiple[~check_multiple['names'].isin(t_names)].copy()
    tutor['tutor_lower'] = tutor['names'].str.lower()
    tutor = tutor[~tutor['tutor_lower'].isin(values)]
    tutor = name_in_title(tutor, title)
    tutor_cont = tutor[tutor['context']==True]
    if(len(tutor_cont)>0):
        return tutor_cont['names'].to_list()[0]
    else:
        tutor = tutor['names'].to_list()[0]
        return tutor
    

def compare_similarity_t(docs, author, threshold = 0.63):
    aux_docs = docs.copy()
    author = nlp_compare(author)
    for name in aux_docs:
        name_aux = nlp_compare(name)
        if author.similarity(name_aux) >= threshold:
            aux_docs.remove(name)
        #print(name, "<->", author, author.similarity(name_aux))
    return aux_docs

def compare_similarity(docs, author):
    author = nlp_compare(author)
    for name in docs:
        name_aux = nlp_compare(name)
        print(name, "<->", author, author.similarity(name_aux))

def compare_names(docs, names, threshold = 0.63):
    aux = docs.copy()
    if len(names) > 1:
        for name in names:
            aux = compare_similarity_t(aux, name, threshold)
        return aux[0]    
    else:
        aux = compare_similarity_t(aux, names[0], threshold)
        return aux[0]
    
def get_persons(data):
    first_data = data
    doc = nlp_person(first_data)
    get_persons_list = []
    for ent in doc.ents:
        if ent.label_ == 'PERSON':
            get_persons_list.append(ent.text)
    return get_persons_list

def get_persons_flair(data):
    sentence = Sentence(data)
    tagger.predict(sentence)
    
    list_of_names = []
    for label in sentence.get_labels('ner'):
        # print label value and score
        if label.value == "PER":
            #print(f'the text of label.data_point is: "{label.data_point.text}"\n')
            list_of_names.append(label.data_point.text)
    return(list_of_names)

def has_tutor_context(start_pos, end_pos, text, word):
    # if the word in this case tutor is nearby the persons name that means it's in the conext and has more chances
    #of beign the tutor
    word_to_check = word.lower()
    #print(text[start_pos-30:end_pos+30])
    if start_pos-30 > 0 and end_pos+30 < len(text):
        if word_to_check in text[start_pos-30:end_pos+30].lower():
            return True
    return False

class Tutor():
    def __init__(self, l):
        self.j = 0
        self.l = l
        
    def compare_all_names(self, file_source, author, threshold = 0.63):
        printProgressBar(self.j, self.l, prefix = 'Progress:', suffix = 'Complete', length = 50)
        try:
            parsed_pdf = parser.from_file(file_source)
            data = parsed_pdf['content']
            data = clean_break_line(data)
            data = clean_spaces_text(data)
            data = clean_points_text(data)
            #data = clean_accents_lower(data)
            #data[:8000]
            docs = get_persons(data[:8000])
            docs = list(map(clean_accents, docs)) #clean the accents

            authors = split_authors(author)
            tutor = compare_names(docs, authors, threshold)
            self.j+=1
            return tutor
        except:
            self.j+=1
            return "No tutor"
        
    def compare_all_names_flair(self, file_source, author, title):
        self.j+=1
        print(file_source)
        printProgressBar(self.j, self.l, prefix = 'Progress:', suffix = 'Complete', length = 50)
        try:
            parsed_pdf = parser.from_file(file_source)
            data = parsed_pdf['content']
            data = clean_break_line(data)
            data = clean_spaces_text(data)
            data = clean_points_text(data)
            #data = clean_commas_text(data)
            #data = clean_accents(data)
            docs = data[:8000]
            docs = clean_accents(docs) #clean the accents
            
            docs_end = data[-6000:]
            docs_end = clean_accents(docs_end) #clean the accents

            #flair
            sentence = Sentence(docs)
            tagger.predict(sentence)
            
            sentence_end = Sentence(docs_end)
            tagger.predict(sentence_end)
            
            
            list_of_names = []
            list_start_pos = []
            list_end_pos = []
            for label in sentence.get_labels('ner'):
                # print label value and score
                if label.value == "PER":
                    #print(f'the text of label.data_point is: "{label.data_point.text}"\n')
                    list_of_names.append(label.data_point.text)
                    list_start_pos.append(label.data_point.start_position)
                    list_end_pos.append(label.data_point.end_position)
            #print(list_of_names)
            
            list_of_names_end = []
            list_start_pos_end = []
            list_end_pos_end = []
            for label in sentence_end.get_labels('ner'):
                # print label value and score
                if label.value == "PER":
                    #print(f'the text of label.data_point is: "{label.data_point.text}"\n')
                    list_of_names_end.append(label.data_point.text)
                    list_start_pos_end.append(label.data_point.start_position)
                    list_end_pos_end.append(label.data_point.end_position)
                    
                    
            df_names = {'names': list_of_names + list_of_names_end,
                        'start_pos': list_start_pos + list_start_pos_end,
                        'end_pos': list_end_pos + list_end_pos_end}
            df_names = pd.DataFrame(df_names)

            df_context = df_names.apply(lambda x: has_tutor_context(x.start_pos, x.end_pos, sentence.text,'tutor'), axis=1)
            df_names['context'] = df_context.to_list()
            
            check_multiple = contains_text_list(df_names['names'].to_list(), split_authors(author), df_names['context'].to_list())
            can_tutor = pick_candidate_tutor(check_multiple, [title])
            #print(can_tutor)
            return can_tutor
        except:
            return "No tutor"
                    
            
            
            
        
        
        
    
    

In [7]:
## names
df_tesis = pd.read_csv('data/thesis_7103_with_resumen_school_complex.csv')
df_tesis

,index,thesis_year,thesis_title,thesis_author,size,thesis_link,pdf_link,resumen,path,exist,isPDF,isScan,school_simple,school_complex
0,1,2009,Estudio de sobretensiones por maniobra debido ...,"De Gregorio S., Luis T.",6.77 MB,http://saber.ucv.ve/handle/10872/17712,http://saber.ucv.ve/bitstream/10872/17712/1/TE...,NaN,thesis_pdf_all/1.pdf,True,True,False,escuela de ingenieria electrica,Escuela de Ingeniería Eléctrica
1,95,20-Jun-2011,Estudio de la influencia de las condiciones de...,"Hernández F., Adamarlys; Jiménez G., Harold",2.46 MB,http://saber.ucv.ve/handle/10872/160,http://saber.ucv.ve/bitstream/10872/160/1/TEG.pdf,NaN,thesis_pdf_all/95.pdf,True,True,False,escuela de ingenieria quimica,Escuela de Ingeniería Química
2,22,31-May-2011,Efecto de los rayos gamma (g) sobre el comport...,"Rodríguez G., Marlene C.",1.62 MB,http://saber.ucv.ve/handle/10872/42,http://saber.ucv.ve/bitstream/10872/42/1/Tesis...,NaN,thesis_pdf_all/22.pdf,True,True,False,escuela de ingenieria quimica,Escuela de Ingeniería Química
3,24,31-May-2011,Determinación de registros pseudo sónicos a pa...,"Urbina P., Eros R.",11.62 MB,http://saber.ucv.ve/handle/10872/53,http://saber.ucv.ve/bitstream/10872/53/1/TRABA...,NaN,thesis_pdf_all/24.pdf,True,True,False,"escuela de geologia, minas y geofisica","Escuela de Geología, Minas y Geofísica"
4,25,31-May-2011,Propuesta de un sistema de administración de i...,"Sánchez M., José R.",562.96 kB,http://saber.ucv.ve/handle/10872/45,http://saber.ucv.ve/bitstream/10872/45/1/TRABA...,NaN,thesis_pdf_all/25.pdf,True,True,False,escuela de ingenieria mecanica,Escuela de Ingeniería Mecanica
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7098,8190,12-Nov-2020,Magnitud y estructura del abastecimiento exter...,"González V., Jorge C.",1.09 MB,http://saber.ucv.ve/handle/10872/20882,http://saber.ucv.ve/bitstream/10872/20882/1/Ma...,NaN,thesis_pdf_all/8190.pdf,True,True,False,escuela de agronomia,Escuela de Agronomía
7099,8192,12-Nov-2020,Evaluación del Impacto de Cambios de Cobertura...,"Rojas N., Ladu G.",3.29 MB,http://saber.ucv.ve/handle/10872/20885,http://saber.ucv.ve/bitstream/10872/20885/1/Tr...,NaN,thesis_pdf_all/8192.pdf,True,True,False,escuela de agronomia,Escuela de Agronomía
7100,8193,18-Nov-2020,Efecto de la sombra sobre la concentración de ...,"Herrera B., Mauro D.",706.17 kB,http://saber.ucv.ve/handle/10872/20888,http://saber.ucv.ve/bitstream/10872/20888/1/Te...,NaN,thesis_pdf_all/8193.pdf,True,True,False,escuela de agronomia,Escuela de Agronomía
7101,8196,18-Nov-2020,Evaluación del comportamiento ecofisiológico d...,"Alfonzo, Miguel",2.14 MB,http://saber.ucv.ve/handle/10872/20891,http://saber.ucv.ve/bitstream/10872/20891/1/TE...,NaN,thesis_pdf_all/8196.pdf,True,True,False,escuela de agronomia,Escuela de Agronomía


In [8]:
df_tesis_100 = df_tesis.copy()
df_tesis_100

,index,thesis_year,thesis_title,thesis_author,size,thesis_link,pdf_link,resumen,path,exist,isPDF,isScan,school_simple,school_complex
0,1,2009,Estudio de sobretensiones por maniobra debido ...,"De Gregorio S., Luis T.",6.77 MB,http://saber.ucv.ve/handle/10872/17712,http://saber.ucv.ve/bitstream/10872/17712/1/TE...,NaN,thesis_pdf_all/1.pdf,True,True,False,escuela de ingenieria electrica,Escuela de Ingeniería Eléctrica
1,95,20-Jun-2011,Estudio de la influencia de las condiciones de...,"Hernández F., Adamarlys; Jiménez G., Harold",2.46 MB,http://saber.ucv.ve/handle/10872/160,http://saber.ucv.ve/bitstream/10872/160/1/TEG.pdf,NaN,thesis_pdf_all/95.pdf,True,True,False,escuela de ingenieria quimica,Escuela de Ingeniería Química
2,22,31-May-2011,Efecto de los rayos gamma (g) sobre el comport...,"Rodríguez G., Marlene C.",1.62 MB,http://saber.ucv.ve/handle/10872/42,http://saber.ucv.ve/bitstream/10872/42/1/Tesis...,NaN,thesis_pdf_all/22.pdf,True,True,False,escuela de ingenieria quimica,Escuela de Ingeniería Química
3,24,31-May-2011,Determinación de registros pseudo sónicos a pa...,"Urbina P., Eros R.",11.62 MB,http://saber.ucv.ve/handle/10872/53,http://saber.ucv.ve/bitstream/10872/53/1/TRABA...,NaN,thesis_pdf_all/24.pdf,True,True,False,"escuela de geologia, minas y geofisica","Escuela de Geología, Minas y Geofísica"
4,25,31-May-2011,Propuesta de un sistema de administración de i...,"Sánchez M., José R.",562.96 kB,http://saber.ucv.ve/handle/10872/45,http://saber.ucv.ve/bitstream/10872/45/1/TRABA...,NaN,thesis_pdf_all/25.pdf,True,True,False,escuela de ingenieria mecanica,Escuela de Ingeniería Mecanica
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7098,8190,12-Nov-2020,Magnitud y estructura del abastecimiento exter...,"González V., Jorge C.",1.09 MB,http://saber.ucv.ve/handle/10872/20882,http://saber.ucv.ve/bitstream/10872/20882/1/Ma...,NaN,thesis_pdf_all/8190.pdf,True,True,False,escuela de agronomia,Escuela de Agronomía
7099,8192,12-Nov-2020,Evaluación del Impacto de Cambios de Cobertura...,"Rojas N., Ladu G.",3.29 MB,http://saber.ucv.ve/handle/10872/20885,http://saber.ucv.ve/bitstream/10872/20885/1/Tr...,NaN,thesis_pdf_all/8192.pdf,True,True,False,escuela de agronomia,Escuela de Agronomía
7100,8193,18-Nov-2020,Efecto de la sombra sobre la concentración de ...,"Herrera B., Mauro D.",706.17 kB,http://saber.ucv.ve/handle/10872/20888,http://saber.ucv.ve/bitstream/10872/20888/1/Te...,NaN,thesis_pdf_all/8193.pdf,True,True,False,escuela de agronomia,Escuela de Agronomía
7101,8196,18-Nov-2020,Evaluación del comportamiento ecofisiológico d...,"Alfonzo, Miguel",2.14 MB,http://saber.ucv.ve/handle/10872/20891,http://saber.ucv.ve/bitstream/10872/20891/1/TE...,NaN,thesis_pdf_all/8196.pdf,True,True,False,escuela de agronomia,Escuela de Agronomía


In [9]:
l = len(df_tesis_100)
l

7103

In [10]:
tutores = Tutor(l)
l = len(df_tesis_100)
compare_fn = np.vectorize(tutores.compare_all_names_flair, otypes=[np.ndarray])
values_c = compare_fn('../'+ df_tesis_100['path'], df_tesis_100['thesis_author'], df_tesis_100['thesis_title'])
df_tesis_100['thesis_tutor'] = values_c.tolist()

2022-08-02 00:16:17,246 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.24/tika-server-1.24.jar to /tmp/tika-server.jar.


../thesis_pdf_all/1.pdf


2022-08-02 00:16:22,547 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.24/tika-server-1.24.jar.md5 to /tmp/tika-server.jar.md5.
2022-08-02 00:16:22,911 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...


../thesis_pdf_all/95.pdf
../thesis_pdf_all/22.pdf-------------------------------------| 0.0% Complete
../thesis_pdf_all/24.pdf-------------------------------------| 0.0% Complete
../thesis_pdf_all/25.pdf-------------------------------------| 0.1% Complete
../thesis_pdf_all/26.pdf-------------------------------------| 0.1% Complete
../thesis_pdf_all/27.pdf-------------------------------------| 0.1% Complete
../thesis_pdf_all/28.pdf-------------------------------------| 0.1% Complete
../thesis_pdf_all/29.pdf-------------------------------------| 0.1% Complete
../thesis_pdf_all/60.pdf-------------------------------------| 0.1% Complete
../thesis_pdf_all/122.pdf------------------------------------| 0.1% Complete
../thesis_pdf_all/61.pdf-------------------------------------| 0.2% Complete
../thesis_pdf_all/63.pdf-------------------------------------| 0.2% Complete
../thesis_pdf_all/96.pdf-------------------------------------| 0.2% Complete
../thesis_pdf_all/20.pdf---------------------------

../thesis_pdf_all/198.pdf------------------------------------| 3.0% Complete
../thesis_pdf_all/199.pdf------------------------------------| 3.0% Complete
../thesis_pdf_all/200.pdf------------------------------------| 3.0% Complete
../thesis_pdf_all/242.pdf------------------------------------| 3.1% Complete
../thesis_pdf_all/244.pdf------------------------------------| 3.1% Complete
../thesis_pdf_all/245.pdf------------------------------------| 3.1% Complete
../thesis_pdf_all/686.pdf------------------------------------| 3.1% Complete
../thesis_pdf_all/688.pdf------------------------------------| 3.1% Complete
../thesis_pdf_all/171.pdf------------------------------------| 3.1% Complete
../thesis_pdf_all/512.pdf------------------------------------| 3.1% Complete
../thesis_pdf_all/316.pdf------------------------------------| 3.2% Complete
../thesis_pdf_all/317.pdf------------------------------------| 3.2% Complete
../thesis_pdf_all/319.pdf------------------------------------| 3.2% Complete

...

../thesis_pdf_all/7084.pdf███████████████████████████--------| 85.9% Complete
../thesis_pdf_all/7072.pdf███████████████████████████--------| 85.9% Complete
../thesis_pdf_all/7057.pdf███████████████████████████--------| 85.9% Complete
../thesis_pdf_all/7056.pdf███████████████████████████--------| 85.9% Complete
../thesis_pdf_all/7008.pdf███████████████████████████--------| 85.9% Complete
../thesis_pdf_all/7006.pdf███████████████████████████--------| 85.9% Complete
../thesis_pdf_all/7141.pdf███████████████████████████--------| 85.9% Complete
../thesis_pdf_all/7181.pdf███████████████████████████--------| 86.0% Complete
../thesis_pdf_all/7109.pdf███████████████████████████--------| 86.0% Complete
../thesis_pdf_all/7110.pdf███████████████████████████--------| 86.0% Complete
../thesis_pdf_all/7111.pdf████████████████████████████-------| 86.0% Complete
../thesis_pdf_all/7112.pdf████████████████████████████-------| 86.0% Complete
../thesis_pdf_all/7113.pdf████████████████████████████-------| 8

../thesis_pdf_all/7235.pdf█████████████████████████████------| 88.8% Complete
../thesis_pdf_all/7236.pdf█████████████████████████████------| 88.8% Complete
../thesis_pdf_all/7237.pdf█████████████████████████████------| 88.8% Complete
../thesis_pdf_all/7238.pdf█████████████████████████████------| 88.9% Complete
../thesis_pdf_all/7269.pdf█████████████████████████████------| 88.9% Complete
../thesis_pdf_all/7270.pdf█████████████████████████████------| 88.9% Complete
../thesis_pdf_all/7272.pdf█████████████████████████████------| 88.9% Complete
../thesis_pdf_all/7305.pdf█████████████████████████████------| 88.9% Complete
../thesis_pdf_all/7306.pdf█████████████████████████████------| 88.9% Complete
../thesis_pdf_all/7307.pdf█████████████████████████████------| 88.9% Complete
../thesis_pdf_all/7308.pdf█████████████████████████████------| 89.0% Complete
../thesis_pdf_all/7309.pdf█████████████████████████████------| 89.0% Complete
../thesis_pdf_all/7310.pdf█████████████████████████████------| 8

../thesis_pdf_all/7554.pdf██████████████████████████████-----| 91.8% Complete
../thesis_pdf_all/7524.pdf██████████████████████████████-----| 91.8% Complete
../thesis_pdf_all/7520.pdf██████████████████████████████-----| 91.8% Complete
../thesis_pdf_all/7584.pdf██████████████████████████████-----| 91.8% Complete
../thesis_pdf_all/7519.pdf██████████████████████████████-----| 91.8% Complete
../thesis_pdf_all/7462.pdf██████████████████████████████-----| 91.8% Complete
../thesis_pdf_all/7463.pdf██████████████████████████████-----| 91.9% Complete
../thesis_pdf_all/7464.pdf██████████████████████████████-----| 91.9% Complete
../thesis_pdf_all/7466.pdf██████████████████████████████-----| 91.9% Complete
../thesis_pdf_all/7467.pdf██████████████████████████████-----| 91.9% Complete
../thesis_pdf_all/7468.pdf██████████████████████████████-----| 91.9% Complete
../thesis_pdf_all/7469.pdf██████████████████████████████-----| 91.9% Complete
../thesis_pdf_all/7471.pdf██████████████████████████████-----| 9

../thesis_pdf_all/7826.pdf████████████████████████████████---| 94.7% Complete
../thesis_pdf_all/7744.pdf████████████████████████████████---| 94.7% Complete
../thesis_pdf_all/7827.pdf████████████████████████████████---| 94.8% Complete
../thesis_pdf_all/7815.pdf████████████████████████████████---| 94.8% Complete
../thesis_pdf_all/7828.pdf████████████████████████████████---| 94.8% Complete
../thesis_pdf_all/7830.pdf████████████████████████████████---| 94.8% Complete
../thesis_pdf_all/7831.pdf████████████████████████████████---| 94.8% Complete
../thesis_pdf_all/7832.pdf████████████████████████████████---| 94.8% Complete
../thesis_pdf_all/7833.pdf████████████████████████████████---| 94.8% Complete
../thesis_pdf_all/7835.pdf████████████████████████████████---| 94.9% Complete
../thesis_pdf_all/7836.pdf████████████████████████████████---| 94.9% Complete
../thesis_pdf_all/7837.pdf████████████████████████████████---| 94.9% Complete
../thesis_pdf_all/7838.pdf████████████████████████████████---| 9

../thesis_pdf_all/8039.pdf█████████████████████████████████--| 97.7% Complete
../thesis_pdf_all/8088.pdf█████████████████████████████████--| 97.7% Complete
../thesis_pdf_all/8076.pdf█████████████████████████████████--| 97.7% Complete
../thesis_pdf_all/8060.pdf█████████████████████████████████--| 97.7% Complete
../thesis_pdf_all/8059.pdf█████████████████████████████████--| 97.7% Complete
../thesis_pdf_all/8058.pdf█████████████████████████████████--| 97.8% Complete
../thesis_pdf_all/8031.pdf█████████████████████████████████--| 97.8% Complete
../thesis_pdf_all/8034.pdf█████████████████████████████████--| 97.8% Complete
../thesis_pdf_all/8035.pdf█████████████████████████████████--| 97.8% Complete
../thesis_pdf_all/8036.pdf█████████████████████████████████--| 97.8% Complete
../thesis_pdf_all/8037.pdf█████████████████████████████████--| 97.8% Complete
../thesis_pdf_all/8038.pdf█████████████████████████████████--| 97.8% Complete
../thesis_pdf_all/8040.pdf█████████████████████████████████--| 9

In [11]:
df_tesis_100.to_csv("./data/thesis_tutor_7103_v3.csv", index=False)

In [10]:
df_tutor = pd.read_csv("./data/thesis_tutor_7103_v3.csv")
df_tutor

,index,thesis_year,thesis_title,thesis_author,size,thesis_link,pdf_link,resumen,path,exist,isPDF,isScan,school_simple,school_complex,thesis_tutor
0,1,2009,Estudio de sobretensiones por maniobra debido ...,"De Gregorio S., Luis T.",6.77 MB,http://saber.ucv.ve/handle/10872/17712,http://saber.ucv.ve/bitstream/10872/17712/1/TE...,NaN,thesis_pdf_all/1.pdf,True,True,False,escuela de ingenieria electrica,Escuela de Ingeniería Eléctrica,Julio Molina
1,95,20-Jun-2011,Estudio de la influencia de las condiciones de...,"Hernández F., Adamarlys; Jiménez G., Harold",2.46 MB,http://saber.ucv.ve/handle/10872/160,http://saber.ucv.ve/bitstream/10872/160/1/TEG.pdf,NaN,thesis_pdf_all/95.pdf,True,True,False,escuela de ingenieria quimica,Escuela de Ingeniería Química,Luis Melo
2,22,31-May-2011,Efecto de los rayos gamma (g) sobre el comport...,"Rodríguez G., Marlene C.",1.62 MB,http://saber.ucv.ve/handle/10872/42,http://saber.ucv.ve/bitstream/10872/42/1/Tesis...,NaN,thesis_pdf_all/22.pdf,True,True,False,escuela de ingenieria quimica,Escuela de Ingeniería Química,Carmen Albano
3,24,31-May-2011,Determinación de registros pseudo sónicos a pa...,"Urbina P., Eros R.",11.62 MB,http://saber.ucv.ve/handle/10872/53,http://saber.ucv.ve/bitstream/10872/53/1/TRABA...,NaN,thesis_pdf_all/24.pdf,True,True,False,"escuela de geologia, minas y geofisica","Escuela de Geología, Minas y Geofísica",Oswaldo Salazar
4,25,31-May-2011,Propuesta de un sistema de administración de i...,"Sánchez M., José R.",562.96 kB,http://saber.ucv.ve/handle/10872/45,http://saber.ucv.ve/bitstream/10872/45/1/TRABA...,NaN,thesis_pdf_all/25.pdf,True,True,False,escuela de ingenieria mecanica,Escuela de Ingeniería Mecanica,Rafael D'AndreaPresentado
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7098,8190,12-Nov-2020,Magnitud y estructura del abastecimiento exter...,"González V., Jorge C.",1.09 MB,http://saber.ucv.ve/handle/10872/20882,http://saber.ucv.ve/bitstream/10872/20882/1/Ma...,NaN,thesis_pdf_all/8190.pdf,True,True,False,escuela de agronomia,Escuela de Agronomía,Agustin Morales
7099,8192,12-Nov-2020,Evaluación del Impacto de Cambios de Cobertura...,"Rojas N., Ladu G.",3.29 MB,http://saber.ucv.ve/handle/10872/20885,http://saber.ucv.ve/bitstream/10872/20885/1/Tr...,NaN,thesis_pdf_all/8192.pdf,True,True,False,escuela de agronomia,Escuela de Agronomía,Oscar Silva
7100,8193,18-Nov-2020,Efecto de la sombra sobre la concentración de ...,"Herrera B., Mauro D.",706.17 kB,http://saber.ucv.ve/handle/10872/20888,http://saber.ucv.ve/bitstream/10872/20888/1/Te...,NaN,thesis_pdf_all/8193.pdf,True,True,False,escuela de agronomia,Escuela de Agronomía,Livia V Pinto S
7101,8196,18-Nov-2020,Evaluación del comportamiento ecofisiológico d...,"Alfonzo, Miguel",2.14 MB,http://saber.ucv.ve/handle/10872/20891,http://saber.ucv.ve/bitstream/10872/20891/1/TE...,NaN,thesis_pdf_all/8196.pdf,True,True,False,escuela de agronomia,Escuela de Agronomía,Liliana Marquez


In [11]:
df_tutor_clean = df_tutor[df_tutor['thesis_tutor']!='No tutor']
df_tutor_clean

,index,thesis_year,thesis_title,thesis_author,size,thesis_link,pdf_link,resumen,path,exist,isPDF,isScan,school_simple,school_complex,thesis_tutor
0,1,2009,Estudio de sobretensiones por maniobra debido ...,"De Gregorio S., Luis T.",6.77 MB,http://saber.ucv.ve/handle/10872/17712,http://saber.ucv.ve/bitstream/10872/17712/1/TE...,NaN,thesis_pdf_all/1.pdf,True,True,False,escuela de ingenieria electrica,Escuela de Ingeniería Eléctrica,Julio Molina
1,95,20-Jun-2011,Estudio de la influencia de las condiciones de...,"Hernández F., Adamarlys; Jiménez G., Harold",2.46 MB,http://saber.ucv.ve/handle/10872/160,http://saber.ucv.ve/bitstream/10872/160/1/TEG.pdf,NaN,thesis_pdf_all/95.pdf,True,True,False,escuela de ingenieria quimica,Escuela de Ingeniería Química,Luis Melo
2,22,31-May-2011,Efecto de los rayos gamma (g) sobre el comport...,"Rodríguez G., Marlene C.",1.62 MB,http://saber.ucv.ve/handle/10872/42,http://saber.ucv.ve/bitstream/10872/42/1/Tesis...,NaN,thesis_pdf_all/22.pdf,True,True,False,escuela de ingenieria quimica,Escuela de Ingeniería Química,Carmen Albano
3,24,31-May-2011,Determinación de registros pseudo sónicos a pa...,"Urbina P., Eros R.",11.62 MB,http://saber.ucv.ve/handle/10872/53,http://saber.ucv.ve/bitstream/10872/53/1/TRABA...,NaN,thesis_pdf_all/24.pdf,True,True,False,"escuela de geologia, minas y geofisica","Escuela de Geología, Minas y Geofísica",Oswaldo Salazar
4,25,31-May-2011,Propuesta de un sistema de administración de i...,"Sánchez M., José R.",562.96 kB,http://saber.ucv.ve/handle/10872/45,http://saber.ucv.ve/bitstream/10872/45/1/TRABA...,NaN,thesis_pdf_all/25.pdf,True,True,False,escuela de ingenieria mecanica,Escuela de Ingeniería Mecanica,Rafael D'AndreaPresentado
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7098,8190,12-Nov-2020,Magnitud y estructura del abastecimiento exter...,"González V., Jorge C.",1.09 MB,http://saber.ucv.ve/handle/10872/20882,http://saber.ucv.ve/bitstream/10872/20882/1/Ma...,NaN,thesis_pdf_all/8190.pdf,True,True,False,escuela de agronomia,Escuela de Agronomía,Agustin Morales
7099,8192,12-Nov-2020,Evaluación del Impacto de Cambios de Cobertura...,"Rojas N., Ladu G.",3.29 MB,http://saber.ucv.ve/handle/10872/20885,http://saber.ucv.ve/bitstream/10872/20885/1/Tr...,NaN,thesis_pdf_all/8192.pdf,True,True,False,escuela de agronomia,Escuela de Agronomía,Oscar Silva
7100,8193,18-Nov-2020,Efecto de la sombra sobre la concentración de ...,"Herrera B., Mauro D.",706.17 kB,http://saber.ucv.ve/handle/10872/20888,http://saber.ucv.ve/bitstream/10872/20888/1/Te...,NaN,thesis_pdf_all/8193.pdf,True,True,False,escuela de agronomia,Escuela de Agronomía,Livia V Pinto S
7101,8196,18-Nov-2020,Evaluación del comportamiento ecofisiológico d...,"Alfonzo, Miguel",2.14 MB,http://saber.ucv.ve/handle/10872/20891,http://saber.ucv.ve/bitstream/10872/20891/1/TE...,NaN,thesis_pdf_all/8196.pdf,True,True,False,escuela de agronomia,Escuela de Agronomía,Liliana Marquez


In [13]:
df_tutor_clean.to_csv("./data/thesis_tutor_7078_clean_v1.csv")

In [271]:
contains_text('fanny ceballos', 'Programa de educación para la prevención del virus del papiloma humano (vph) dirigido a las usuarias que asisten a la consulta de ginecología del Hospital General del Oeste “Dr. José Gregorio Hernández” de los Magallanes de Catia en el tercer trimestre del año 2005.')

['fanny', 'ceballos']
['Programa', 'de', 'educación', 'para', 'la', 'prevención', 'del', 'virus', 'del', 'papiloma', 'humano', '(vph)', 'dirigido', 'a', 'las', 'usuarias', 'que', 'asisten', 'a', 'la', 'consulta', 'de', 'ginecología', 'del', 'Hospital', 'General', 'del', 'Oeste', '“Dr.', 'José', 'Gregorio', 'Hernández”', 'de', 'los', 'Magallanes', 'de', 'Catia', 'en', 'el', 'tercer', 'trimestre', 'del', 'año', '2005.']


False

In [80]:
df_tesis_100['thesis_tutor']

0    Julio Molina
Name: thesis_tutor, dtype: object

In [11]:
df_tesis_100.to_csv("./data/thesis_tutor_5.csv", index=False)

In [42]:
thesis_number = '6'
file_source = '../'+'thesis_pdf_all/' + thesis_number + '.pdf'

parsed_pdf = parser.from_file(file_source)
data = parsed_pdf['content']
data = clean_break_line(data)
data = clean_spaces_text(data)
data = clean_points_text(data)
data = clean_commas_text(data)
data = clean_accents(data)
data[:8000]

'TRABAJO ESPECIAL DE GRADO TRABAJO ESPECIAL DE GRADO ESTUDIO DE LA INFLUENCIA DE LAS CONDICIONES DE PREPARACION DE CATALIZADORES DE Ru/ZEOLITA-Y Y Ru/ Al2 O3 EN REACCIONES DE HIDROTRATAMIENTO Presentado ante la Ilustre Universidad Central de Venezuela para optar al Titulo de Ingeniero Quimico por el Br Fernandez M Fernando J Caracas agosto de 2001 TRABAJO ESPECIAL DE GRADO ESTUDIO DE LA INFLUENCIA DE LAS CONDICIONES DE PREPARACION DE CATALIZADORES DE Ru/ZEOLITA-Y Y Ru/Al2O3 EN REACCIONES DE HIDROTRATAMIENTO Tutores academicos: Prof: Luis Melo Prof: Jose Ruggiero Presentado ante la Ilustre Universidad Central de Venezuela para optar al Titulo de Ingeniero Quimico por el Br Fernandez M Fernando J Caracas agosto de 2001 (c) Fernando Fernandez 2001 Hecho el Deposito Ley Deposito de Legal lft4872001620118 Caracas 10 de agosto de 2001 Los abajo firmantes miembros del jurado designado por el consejo de Escuela de Ingenieria Quimica para evaluar el Trabajo Especial de grado presentado por el B

In [43]:
author = df_tesis[df_tesis['index'] == int(thesis_number)]
author = author['thesis_author']
author = author.to_list()
author = split_authors(author[0])
author

['Fernández, Fernando J.']

In [44]:
title = df_tesis[df_tesis['index'] == int(thesis_number)]
title = title['thesis_title'].to_list()
title = title
title

['Estudio de la influencia de las condiciones de preparación de catalizadores de ru/zeolita-y y ru/ al2 o3 en reacciones de hidrotratamiento']

In [45]:
data_end = data[-6000:]
data_end

' T G Harvey y W Matheson J Catal Vol 101 Pag 253-261 (1986) (3) Y J Kuo y B J Tatarchuk J Catal Vol 112 Pag 229-249 (1988) (4) Y J Kuo R A Cocoo y B Tatarchuk J Catal Vol 112 Pag 250-266 (1988) (5) J Leglise A Janin J Lavavlley y Y Cornet J Catal Vol 114 Pag 388-397 (1988) (6) J Marc Ledoux y D Brahim J Catal Vol 115 Pag 580-590 (1989) (7) J A De los Reyes S Gobolos M Vrinat y M Breysse Catal Lett Vol 5 Pag 17 (1990) (8) J A De los Reyes C Vrinat Geantet y M Breysse Catal Today Vol 10 Pag 645 (1991) (9) R Cid J Neira J M Palacios y Lopez Agudo Appl Catal Vol 125 Pag 169-183 (1995) (10) V Kuogionas M Cattenot J Zotin Portefaix y M Breysse Appl Catal Vol 124 Pag 153-164 (1995) 89 (11) C Milone G Neri M Donato G Musolino y L Mercante Vol 159 Pag 253-258 (1996) (12) M Sugioka F Sado Y Matsumoto y N Maesaki Catal Today Vol 29 Pag 255-259 (1996) (13) M Mhaouer J Lemberton y G Perot Catal Today Pag 241-244 (1996) (14) B Moraweck  G Bergeret M Cattenot  Kuogionas y C Geantet J Catal Pag 45-56

In [46]:
sentence = Sentence(data[:8000])
sentence_end = Sentence(data_end)

In [47]:
tagger.predict(sentence)
tagger.predict(sentence_end)

In [48]:
print('The following NER tags are found:')
# iterate over entities and print
for entity in sentence.get_spans('ner'):
    #print(entity.value)
    print(entity)

The following NER tags are found:
Span[35:40]: "Ilustre Universidad Central de Venezuela" → ORG (1.0)
Span[50:52]: "Fernandez M" → PER (0.9981)
Span[52:54]: "Fernando J" → PER (0.9999)
Span[54:55]: "Caracas" → LOC (1.0)
Span[90:92]: "Luis Melo" → PER (1.0)
Span[94:96]: "Jose Ruggiero" → PER (1.0)
Span[99:104]: "Ilustre Universidad Central de Venezuela" → ORG (1.0)
Span[114:116]: "Fernandez M" → PER (0.9966)
Span[116:118]: "Fernando J" → PER (1.0)
Span[118:119]: "Caracas" → LOC (1.0)
Span[125:127]: "Fernando Fernandez" → PER (1.0)
Span[136:137]: "Caracas" → LOC (1.0)
Span[153:157]: "Escuela de Ingenieria Quimica" → ORG (1.0)
Span[168:172]: "Fernando J Fernandez M" → PER (0.9996)
Span[242:244]: "Carlos Scott" → PER (1.0)
Span[246:248]: "Samir Marsuka" → PER (1.0)
Span[258:260]: "Luis Melo" → PER (1.0)
Span[262:264]: "Jose Ruggiero" → PER (1.0)
Span[274:278]: "Fernandez M Fernando J" → PER (0.9925)
Span[305:307]: "Luis Melo" → PER (1.0)
Span[307:309]: "Jose Ruggiero" → PER (1.0)
Span[310:

In [49]:
print('The following NER tags are found:')
# iterate over entities and print
for entity in sentence_end.get_spans('ner'):
    #print(entity.value)
    print(entity)

The following NER tags are found:
Span[0:3]: "T G Harvey" → PER (1.0)
Span[4:6]: "W Matheson" → PER (1.0)
Span[6:8]: "J Catal" → ORG (0.972)
Span[18:21]: "Y J Kuo" → PER (1.0)
Span[22:25]: "B J Tatarchuk" → PER (1.0)
Span[25:27]: "J Catal" → ORG (0.9198)
Span[37:40]: "Y J Kuo" → PER (1.0)
Span[40:43]: "R A Cocoo" → PER (0.9999)
Span[44:46]: "B Tatarchuk" → PER (1.0)
Span[46:48]: "J Catal" → MISC (0.6946)
Span[58:64]: "J Leglise A Janin J Lavavlley" → PER (0.913)
Span[65:67]: "Y Cornet" → PER (1.0)
Span[67:69]: "J Catal" → ORG (0.7933)
Span[79:82]: "J Marc Ledoux" → PER (1.0)
Span[83:85]: "D Brahim" → PER (1.0)
Span[85:87]: "J Catal" → ORG (0.9844)
Span[97:102]: "J A De los Reyes" → PER (0.9999)
Span[102:104]: "S Gobolos" → PER (1.0)
Span[104:106]: "M Vrinat" → PER (1.0)
Span[107:109]: "M Breysse" → PER (1.0)
Span[109:111]: "Catal Lett" → ORG (0.998)
Span[121:126]: "J A De los Reyes" → PER (0.9999)
Span[126:129]: "C Vrinat Geantet" → PER (1.0)
Span[130:132]: "M Breysse" → PER (1.0)
Span

In [50]:
list_of_names_end = []
list_start_pos_end = []
list_end_pos_end = []
for label in sentence_end.get_labels('ner'):
    # print label value and score
    if label.value == "PER":
        #print(f'the text of label.data_point is: "{label.data_point.text}"\n')
        list_of_names_end.append(label.data_point.text)
        list_start_pos_end.append(label.data_point.start_position)
        list_end_pos_end.append(label.data_point.end_position)
print(list_of_names_end)
print(list_start_pos_end)
print(list_end_pos_end)
#print(sentence.text[352:409])
    #print(f'label.value is: "{label.value}"')
    #print(f'label.score is: "{label.score}"')
    # access the data point to which label attaches and print its text
    #print(f'the text of label.data_point is: "{label.data_point.text}"\n')

['T G Harvey', 'W Matheson', 'Y J Kuo', 'B J Tatarchuk', 'Y J Kuo', 'R A Cocoo', 'B Tatarchuk', 'J Leglise A Janin J Lavavlley', 'Y Cornet', 'J Marc Ledoux', 'D Brahim', 'J A De los Reyes', 'S Gobolos', 'M Vrinat', 'M Breysse', 'J A De los Reyes', 'C Vrinat Geantet', 'M Breysse', 'R Cid', 'J Neira', 'J M Palacios', 'Lopez Agudo', 'V Kuogionas', 'M Cattenot', 'J Zotin Portefaix', 'M Breysse', 'C Milone', 'G Neri', 'M Donato G Musolino', 'L Mercante', 'M Sugioka', 'F Sado Y Matsumoto', 'N Maesaki', 'M Mhaouer', 'J Lemberton', 'G Perot', 'B Moraweck', 'G Bergeret', 'M Cattenot Kuogionas', 'C Geantet', 'Li Di N Ichikuni', 'S Shimazu', 'T Uematsu', 'Ruggiero Jose', 'Giannetto Giuseppe', 'Giannetto Giuseppe', 'Kird-Otmher', 'Giannetto Giuseppe', 'N Charles Satterfield', 'W Alarcon', 'M A Diaz', 'R A Cid', 'Gomez Carolina', 'B Gates J Katzer', 'Y Schuit', 'R Alcala N Gedaias', 'Martha C Matjushin', 'H Shells', 'W Bohringer', 'H Shulz M Shom', 'N R Rahman', 'H E Barbosa', 'E Gonzales', 'J C Af

In [51]:
list_of_names = []
list_start_pos = []
list_end_pos = []
for label in sentence.get_labels('ner'):
    # print label value and score
    if label.value == "PER":
        #print(f'the text of label.data_point is: "{label.data_point.text}"\n')
        list_of_names.append(label.data_point.text)
        list_start_pos.append(label.data_point.start_position)
        list_end_pos.append(label.data_point.end_position)
print(list_of_names)
print(list_start_pos)
print(list_end_pos)
#print(sentence.text[352:409])

['Fernandez M', 'Fernando J', 'Luis Melo', 'Jose Ruggiero', 'Fernandez M', 'Fernando J', 'Fernando Fernandez', 'Fernando J Fernandez M', 'Carlos Scott', 'Samir Marsuka', 'Luis Melo', 'Jose Ruggiero', 'Fernandez M Fernando J', 'Luis Melo', 'Jose Ruggiero', 'Carlos Scott', 'Jose Ruggiero', 'Luis Melo', 'Jose Miguel Fernandez', 'Tomas', 'Adriana Emir Escalona Yelitza', 'Yomaira', 'Guillermo', 'Marco Colmenares', 'Tabata Gil', 'Nury Guaran', 'David Goni', 'Jose Rodriguez', 'Gianna Corrales', 'Carmen Elsa', 'Ibelisse Frayer']
[303, 315, 538, 554, 680, 692, 730, 1008, 1464, 1483, 1574, 1589, 1648, 1832, 1842, 3234, 3506, 3806, 4213, 4321, 4862, 4894, 5126, 5214, 5372, 5385, 5453, 5477, 5707, 5723, 5735]
[314, 325, 547, 567, 691, 702, 748, 1030, 1476, 1496, 1583, 1602, 1670, 1841, 1855, 3246, 3519, 3815, 4234, 4326, 4891, 4901, 5135, 5230, 5382, 5396, 5463, 5491, 5722, 5734, 5750]


In [52]:
df_names = {'names': list_of_names + list_of_names_end, 'start_pos': list_start_pos + list_start_pos_end, 'end_pos': list_end_pos + list_end_pos_end}
df_names = pd.DataFrame(df_names)
df_names

,names,start_pos,end_pos
0,Fernandez M,303,314
1,Fernando J,315,325
2,Luis Melo,538,547
3,Jose Ruggiero,554,567
4,Fernandez M,680,691
...,...,...,...
94,J C Afonso,3015,3025
95,J L Zotin,3089,3098
96,Barrrer,3226,3233
97,Breck,3898,3903


In [53]:
df_context = df_names.apply(lambda x: has_tutor_context(x.start_pos, x.end_pos, sentence.text,'tutor'), axis=1)
df_names['context'] = df_context.to_list()
df_names

,names,start_pos,end_pos,context
0,Fernandez M,303,314,False
1,Fernando J,315,325,False
2,Luis Melo,538,547,True
3,Jose Ruggiero,554,567,False
4,Fernandez M,680,691,False
...,...,...,...,...
94,J C Afonso,3015,3025,False
95,J L Zotin,3089,3098,False
96,Barrrer,3226,3233,False
97,Breck,3898,3903,False


In [54]:
#compare_similarity(list_of_names, author[1])

In [55]:
#compare_names(list_of_names, author, threshold = 0.43)

In [56]:
check_multiple = contains_text_list(df_names['names'].to_list(), author, df_names['context'].to_list())
check_multiple

,names,author,author_lower,contains,context
0,Fernandez M,"Fernández, Fernando J.","fernandez, fernando j.",True,False
1,Fernando J,"Fernández, Fernando J.","fernandez, fernando j.",True,False
2,Luis Melo,"Fernández, Fernando J.","fernandez, fernando j.",False,True
3,Jose Ruggiero,"Fernández, Fernando J.","fernandez, fernando j.",False,False
4,Fernandez M,"Fernández, Fernando J.","fernandez, fernando j.",True,False
...,...,...,...,...,...
94,J C Afonso,"Fernández, Fernando J.","fernandez, fernando j.",False,False
95,J L Zotin,"Fernández, Fernando J.","fernandez, fernando j.",False,False
96,Barrrer,"Fernández, Fernando J.","fernandez, fernando j.",False,False
97,Breck,"Fernández, Fernando J.","fernandez, fernando j.",False,False


In [57]:
list_of_names = df_names['names'].to_list()

In [58]:
check_multiple = contains_text_list(df_names['names'].to_list(), author, df_names['context'].to_list())
can_tutor = pick_candidate_tutor(check_multiple, title)
can_tutor

'Luis Melo'

In [22]:
can_tutor['names'].to_list()[0]

TypeError: string indices must be integers

In [671]:
compare_names(can_tutor['names'].to_list(), author, threshold = 0.5)

'Aura Perez'

In [663]:
can_tutor['is_in_title'] = can_tutor['tutor_lower'].apply(contains_text, args = title)
can_tutor

,names,author,author_lower,contains,tutor_lower,is_in_title
6,Aura Perez,"Carrillo C., Nancy","carrillo c., nancy",False,aura perez,False
7,Jaime Garrido,"Carrillo C., Nancy","carrillo c., nancy",False,jaime garrido,False
10,Mylanyela Rosana,"Carrillo C., Nancy","carrillo c., nancy",False,mylanyela rosana,False
11,Omar,"Carrillo C., Nancy","carrillo c., nancy",False,omar,False
13,Venus,"Carrillo C., Nancy","carrillo c., nancy",False,venus,False
16,Aura Perez,"Carrillo C., Nancy","carrillo c., nancy",False,aura perez,False
17,Jaime Garrido,"Carrillo C., Nancy","carrillo c., nancy",False,jaime garrido,False
22,Aura Perez,"Carrillo C., Nancy","carrillo c., nancy",False,aura perez,False
23,Jaime Garrido,"Carrillo C., Nancy","carrillo c., nancy",False,jaime garrido,False
30,Aura Perez,"Márquez S., Migdalia","marquez s., migdalia",False,aura perez,False


In [566]:
contains_text("INGRID PEREZ", title)

False

In [503]:
compare_names(can_tutor['names'].to_list(), author)

'INGRID PEREZ'

In [345]:
author[1].split(" ")

['', 'Contreras,', 'Rosa', 'Y.']

In [346]:
list_of_names[0].split(" ")

['JOSE', 'GREGORIO', 'HERNANDEZ']

In [347]:

contains_text(list_of_names[0], author[0])

False

In [348]:
pd.set_option('display.max_rows', None)
check_multiple = contains_text_list(list_of_names, author)

In [349]:
check_multiple_names = check_multiple['names']
#check_multiple_names.to_list()

In [350]:
true_values = check_multiple[check_multiple["contains"]==True]
t_names = true_values["names"]
t_names.to_list()

['Colmenares Suleimy',
 'Colmenares Suleimy',
 'Colmenares Suleimy',
 'Colmenares Suleimy Contreras',
 'Colmenares Suleimy Contreras',
 'Contreras Rosa Yadira',
 'Contreras Rosa Yadira',
 'Rosa Yadira',
 'Colmenares Suleimy Contreras',
 'Rosa Yadira',
 'Colmenares Suleimy Contreras',
 'Rosa Yadira Valera',
 'Valera Carmen Elena',
 'Valera Carmen Elena',
 'Carmen Valera',
 'Valera Carmen Elena',
 'Rosa Yadira Valera',
 'Carmen Elena']

In [361]:
tutor = check_multiple[~check_multiple['names'].isin(t_names.to_list())]
tutor = tutor[lower(tutor["names"]) != "JOSE GREGORIO HERNANDEZ"]
tutor[tutor["names"] != "Dios"]

NameError: name 'lower' is not defined

In [48]:
first_data = data[:8000]
doc = nlp_person(first_data)
get_persons = []
for ent in doc.ents:
    if ent.label_ == 'PERSON':
        get_persons.append(ent.text)
    
get_persons

['Rosa Yadira Valera',
 'Colmenares Suleimy',
 'Contreras Rosa Yadira',
 'Valera Carmen Elena',
 'Rosa Yadira',
 'Dios',
 'Yadira',
 'Valera Carmen Elena Para',
 'Rosa Yadira Valera',
 'Carmen Elena']

In [49]:
get_persons = list(map(clean_accents_lower, get_persons))

In [57]:
## names
df_tesis = pd.read_csv('data/thesis_7149_with_resumen_school_complex.csv')
authors = df_tesis[df_tesis['index'] == int(thesis_number)]['thesis_author']
authors
authors

4    Colmenares, Suleimy; Contreras, Rosa Y.; Valer...
Name: thesis_author, dtype: object

In [59]:
split_fn = np.vectorize(split_authors)
split_fn(authors).tolist()

ValueError: setting an array element with a sequence

In [465]:
split_authors(df_tesis['thesis_author'][1])

['González P., Héctor V.']

In [466]:
df_tesis['thesis_author']

0                                 De Gregorio S., Luis T.
1                                  González P., Héctor V.
2                Carrillo C., Nancy; Márquez S., Migdalia
3          Casaña, Yurima; Florido, Lisbeth; Nieves, Rosa
4       Colmenares, Suleimy; Contreras, Rosa Y.; Valer...
                              ...                        
7144                              Sandia I., Alexandra C.
7145                            Temprano G., Alejandro J.
7146                                  Landaeta R., Ana G.
7147                                Borges F., Gabriel A.
7148                          Utrera Ávila, Joselin Marly
Name: thesis_author, Length: 7149, dtype: object

In [467]:
tutores = Tutor

In [469]:
compare_fn = np.vectorize(tutores.compare_all_names)
compare_fn()

In [435]:
top10_clean[int(thesis_number)-1]

'Adrian Mogollón Julia; Echezuria Perozo Jorge E; Valero Goncalves Yusmary L'

In [436]:
names = list(map(clean_accents_lower, split_authors(top10_clean[int(thesis_number)-1])))
names 

['Adrian Mogollon Julia',
 ' Echezuria Perozo Jorge E',
 ' Valero Goncalves Yusmary L']

In [437]:
names = split_authors(top10_clean[int(thesis_number)-1])
names

['Adrian Mogollón Julia',
 ' Echezuria Perozo Jorge E',
 ' Valero Goncalves Yusmary L']

In [438]:
compare_similarity(get_persons, names[0])

Yusmary TUTOR <-> Adrian Mogollón Julia 0.11692980703110051
Angel Moros R. <-> Adrian Mogollón Julia 0.45962000177794815
Echezuria <-> Adrian Mogollón Julia 0.0
Yusmary <-> Adrian Mogollón Julia 0.0
Moros <-> Adrian Mogollón Julia 0.15718233851904745
Angel ivDEDICATORIA <-> Adrian Mogollón Julia 0.37152739068934554
Mil Gracias <-> Adrian Mogollón Julia 0.26608767915043785
Julita <-> Adrian Mogollón Julia 0.0
Yusmy <-> Adrian Mogollón Julia 0.0
Yusmary vAGRADECIMIENTOS <-> Adrian Mogollón Julia 0.0
Carmen Barrios <-> Adrian Mogollón Julia 0.5507273551865355
Profesor Angel Moros <-> Adrian Mogollón Julia 0.41511536453209774
Aida Damas <-> Adrian Mogollón Julia 0.34371654321983364
Angel Moros R. RESUMEN <-> Adrian Mogollón Julia 0.459620014454642


/tmp/ipykernel_5742/1030706800.py:5: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  print(name, "<->", author, author.similarity(name_aux))


In [439]:
compare_names(get_persons, names, threshold=.46)

/tmp/ipykernel_5742/397893262.py:28: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  if author.similarity(name_aux) >= threshold:


'Yusmary TUTOR'

In [440]:
fn_compare_names = np.vectorize(compare_names)

In [ ]:
values_compare = fn_compare_names

In [431]:
compare_names(get_persons, names)

/tmp/ipykernel_5742/397893262.py:28: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  if author.similarity(name_aux) >= threshold:


'Yusmary TUTOR'

In [407]:
compare_names(get_persons, names)

/tmp/ipykernel_5742/397893262.py:28: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  if author.similarity(name_aux) >= threshold:


'Yusmary TUTOR'

In [116]:
compare_similarity_t(get_persons, names[1])

/tmp/ipykernel_5742/1498625836.py:27: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  if author.similarity(name_aux) >= threshold:


['Angel Moros R',
 'Yusmary Valero G',
 'Yusmy',
 'Alejandra Gonzalez',
 'Angel Moros',
 'Yusmary Valero',
 'Angel Moros R RESUMEN']

In [117]:
compare_similarity(get_persons, names[2])

Angel Moros R <->  Valero Goncalves Yusmary L 0.32039604916347836
Yusmary Valero G <->  Valero Goncalves Yusmary L 0.6697135616296339
Yusmy <->  Valero Goncalves Yusmary L 0.0
Alejandra Gonzalez <->  Valero Goncalves Yusmary L 0.47268456182808843
Angel Moros <->  Valero Goncalves Yusmary L 0.25236357407492055
Yusmary Valero <->  Valero Goncalves Yusmary L 0.7156886585504478
Angel Moros R RESUMEN <->  Valero Goncalves Yusmary L 0.3203960671539872


/tmp/ipykernel_5742/1030706800.py:5: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  print(name, "<->", author, author.similarity(name_aux))


In [118]:
compare_similarity_t(get_persons, names[2])

['Angel Moros R',
 'Yusmy',
 'Alejandra Gonzalez',
 'Angel Moros',
 'Angel Moros R RESUMEN']

In [40]:
#file_source = '../'+'thesis_pdf_all/1.pdf'

#parsed_pdf = parser.from_file(file_source)
#data = parsed_pdf['content']

#data

In [42]:
#data = data.replace('\n',"")
#data = clean_spaces_text(data)
#data

In [43]:
#first_data = data[:600]
#first_data

In [8]:
doc = nlp_spa("Margarita camina sobre la torta")

for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Margarita camina 0 16 PERSON


In [9]:
doc = nlp_spa(first_data)

for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Luis T. 288 295 PERSON
2009 352 356 DATE
CONSTANCIA DE APROBACIÓN 360 384 PERSON
13 de mayo de 2009 394 412 DATE
Luis T. De Greg 585 600 PERSON


In [23]:
#nlp_spa = spacy.load('en_core_web_sm')
#nlp_spa = spacy.load('es_core_news_sm')
#nlp_spa = spacy.load('en_core_web_sm')
#nlp_person = spacy.load('es_core_news_lg')
nlp_compare = spacy.load('en_core_web_lg')
nlp_person = spacy.load('en_core_web_trf')
#nlp_spa = spacy.load('es_dep_news_trf')

In [24]:
thesis_number = '3'
file_source = '../'+'thesis_pdf_all/' + thesis_number + '.pdf'

parsed_pdf = parser.from_file(file_source)
data = parsed_pdf['content']
data = clean_break_line(data)
data = clean_spaces_text(data)
data = clean_points_text(data)
data[:8000]

'UNIVERSIDAD CENTRAL DE VENEZUELAUNIVERSIDAD CENTRAL DE VENEZUELA FACULTAD DE HUMANIDADES Y EDUCACIÓN ESCUELA DE BIBLIOTECOLOGÍA Y ARCHIVOLOGÍA LA PRODUCCIÓN DE CONOCIMIENTO: UN ANÁLISIS MÉTRICO EN EL PROGRAMA DE INVESTIGACIÓN Y DESARROLLO DEL IDEC-FAU-UCV Trabajo de licenciatura presentado como requisito de la Escuela de Bibliotecología y Archivología de la Universidad Central de Venezuela para optar al Título de Licenciadas en Bibliotecología y Licenciado en Archivología PRESENTADO: Adrián M Julia Echezuría P Jorge Valero G Yusmary TUTOR: Prof Dr Ángel Moros R Caracas octubre 2008 ADRIÁN Julia Variabilidad de productores y producción de conocimiento del Programa de Investigación y Desarrollo del IDEC-FAU-UCV / Julia Adrián M Jorge Echezuría P Yusmary Valero G ; Tutor Prof Dr Ángel Moros R— Caracas 2008 x 107 h; il ; 30cm Trabajo de Licenciatura (Bibliotecología y Archivología) – Universidad Central de Venezuela 2008 1 ANALISIS MÉTRICO 2 BIBLIOMETRÍA 3 PRODUCCIÓN DE CONOCIMIENTO 4 PID

In [25]:
first_data = data[:8000]
doc = nlp_person(first_data)

for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Adrián M 489 497 PERSON
Julia Echezuría 498 513 PERSON
Jorge Valero G Yusmary TUTOR 516 544 PERSON
Ángel Moros R 554 567 PERSON
Yusmary Valero G 754 770 PERSON
Universidad Central de Venezuela 893 925 ORG
Yusmy 1544 1549 PERSON
Jorge 1764 1769 PERSON
Alejandra González 2140 2158 PERSON
Carmen Barrios 2198 2212 PERSON
Ángel Moros 2266 2277 PERSON
José López 2410 2420 PERSON
Isabel Bello 2664 2676 PERSON
Instituto de Desarrollo Experimental de la Construcción ( 3943 4000 ORG
Programa de Investigación y Desarrollo ( 4013 4053 ORG
1985-2007 5702 5711 DATE
2000-2007 5777 5786 DATE
1985-2007 5864 5873 DATE
2000-2007 5952 5961 DATE
1985-2007 6018 6027 DATE
Julia Adrián Jorge Echezuria 6465 6493 PERSON
Yusmary Valero 6494 6508 PERSON
Ángel Moros R RESUMEN 6516 6537 PERSON
2000-2007 6734 6743 DATE
1985-2006 6817 6826 DATE
2 6967 6968 CARDINAL
3 6999 7000 CARDINAL
two 7513 7516 CARDINAL
2000-2007 7591 7600 DATE
1995-2006 7677 7686 DATE
three 7709 7714 CARDINAL
1 7723 7724 CARDINAL
2 7816 7817 CA

In [27]:
get_persons = []
for ent in doc.ents:
    if ent.label_ == 'PERSON':
        get_persons.append(ent.text)
    
get_persons

['Adrián M',
 'Julia Echezuría',
 'Jorge Valero G Yusmary TUTOR',
 'Ángel Moros R',
 'Yusmary Valero G',
 'Yusmy',
 'Jorge',
 'Alejandra González',
 'Carmen Barrios',
 'Ángel Moros',
 'José López',
 'Isabel Bello',
 'Julia Adrián Jorge Echezuria',
 'Yusmary Valero',
 'Ángel Moros R RESUMEN']

In [28]:
## names
df_tesis = pd.read_csv('data/url_thesis_8211_with_pdf_scan.csv')
df_tesis

,index,thesis_year,thesis_title,thesis_author,size,thesis_link,pdf_link,resumen,path,exist,isPDF
0,1,2009,Estudio de sobretensiones por maniobra debido ...,"De Gregorio S., Luis T.",6.77 MB,http://saber.ucv.ve/handle/10872/17712,http://saber.ucv.ve/bitstream/10872/17712/1/TE...,NaN,thesis_pdf_all/1.pdf,True,True
1,2,2010,La Economía del Ocio como una alternativa de e...,"Rivera, Elizabeth",2 MB,http://saber.ucv.ve/handle/10872/16245,http://saber.ucv.ve/bitstream/10872/16245/1/TG...,NaN,thesis_pdf_all/2.pdf,True,True
2,3,27-May-2011,La producción de conocimiento : un análisis mé...,"Adrian Mogollón, Julia; Echezuria Perozo, Jorg...",995.92 kB,http://saber.ucv.ve/handle/10872/29,http://saber.ucv.ve/bitstream/10872/29/1/TL-EB...,NaN,thesis_pdf_all/3.pdf,True,True
3,4,27-May-2011,Desarrollo de un algoritmo computacional para ...,"D'Amico, Walter F.",506.32 kB,http://saber.ucv.ve/handle/10872/32,http://saber.ucv.ve/bitstream/10872/32/1/Tesis...,NaN,thesis_pdf_all/4.pdf,True,True
4,5,27-May-2011,Evaluación geofísica integral de los conos alu...,"Cragno, Analia V.; Romero, Marcos A.",3.02 MB,http://saber.ucv.ve/handle/10872/31,http://saber.ucv.ve/bitstream/10872/31/1/Evalu...,NaN,thesis_pdf_all/5.pdf,True,True
...,...,...,...,...,...,...,...,...,...,...,...
8168,8207,4-Dec-2020,Elaboración de un manual de procedimientos e i...,"Armas R., Rodolfo J.",924.37 kB,http://saber.ucv.ve/handle/10872/20906,http://saber.ucv.ve/bitstream/10872/20906/1/TE...,NaN,thesis_pdf_all/8207.pdf,True,True
8169,8208,8-Mar-2021,Condiciones de vida de los estudiantes de Geoq...,"Pérez, Andrea; Pinto, Hendrith",7.21 MB,http://saber.ucv.ve/handle/10872/20969,http://saber.ucv.ve/bitstream/10872/20969/1/CO...,NaN,thesis_pdf_all/8208.pdf,True,True
8170,8209,8-Mar-2021,Análisis de los elementos pospornográficos de ...,"Belandria, Sofía",1.13 MB,http://saber.ucv.ve/handle/10872/20970,http://saber.ucv.ve/bitstream/10872/20970/1/So...,NaN,thesis_pdf_all/8209.pdf,True,True
8171,8210,11-Mar-2021,La Fecundidad Adolescente en Venezuela vista a...,"Simao, Edith Carolina",3.56 MB,http://saber.ucv.ve/handle/10872/20973,http://saber.ucv.ve/bitstream/10872/20973/1/TF...,NaN,thesis_pdf_all/8210.pdf,True,True


In [29]:
top10_authors = df_tesis['thesis_author'][:10]
top10_authors = top10_authors.values.tolist()
top10_authors

['De Gregorio S., Luis T.',
 'Rivera, Elizabeth',
 'Adrian Mogollón, Julia; Echezuria Perozo, Jorge E.; Valero Goncalves, Yusmary L.',
 "D'Amico, Walter F.",
 'Cragno, Analia V.; Romero, Marcos A.',
 'Fernández, Fernando J.',
 'Echezuria Perozo, Jorge E.; Rondon, María C.; Gutierrez Luna, Melany',
 'Silva P., Daniel J.',
 'Silva, Carolina del V.',
 'Suárez B., Efren J.']

In [30]:
top10_authors[2]

'Adrian Mogollón, Julia; Echezuria Perozo, Jorge E.; Valero Goncalves, Yusmary L.'

In [31]:
vec_points = np.vectorize(clean_points_text) 

In [32]:
top10_clean = vec_points(top10_authors)
top10_clean = top10_clean.tolist()

In [107]:
def compare_similarity(docs, author):
    author = nlp_compare(author)
    for name in docs:
        name_aux = nlp_compare(name)
        print(name, "<->", author, author.similarity(name_aux))

In [34]:
compare_similarity(get_persons, str(top10_clean[2]))

Adrián M <-> Adrian Mogollón Julia; Echezuria Perozo Jorge E; Valero Goncalves Yusmary L 0.33937008053670276
Julia Echezuría <-> Adrian Mogollón Julia; Echezuria Perozo Jorge E; Valero Goncalves Yusmary L 0.5670012875522249
Jorge Valero G Yusmary TUTOR <-> Adrian Mogollón Julia; Echezuria Perozo Jorge E; Valero Goncalves Yusmary L 0.7070172418471509
Ángel Moros R <-> Adrian Mogollón Julia; Echezuria Perozo Jorge E; Valero Goncalves Yusmary L 0.40224459980277893
Yusmary Valero G <-> Adrian Mogollón Julia; Echezuria Perozo Jorge E; Valero Goncalves Yusmary L 0.5975819452444061
Yusmy <-> Adrian Mogollón Julia; Echezuria Perozo Jorge E; Valero Goncalves Yusmary L 0.0
Jorge <-> Adrian Mogollón Julia; Echezuria Perozo Jorge E; Valero Goncalves Yusmary L 0.676725298944286
Alejandra González <-> Adrian Mogollón Julia; Echezuria Perozo Jorge E; Valero Goncalves Yusmary L 0.35765417137227434
Carmen Barrios <-> Adrian Mogollón Julia; Echezuria Perozo Jorge E; Valero Goncalves Yusmary L 0.47139572

/tmp/ipykernel_5742/1030706800.py:5: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  print(name, "<->", author, author.similarity(name_aux))


In [35]:
top10_clean[2].split(";")

['Adrian Mogollón Julia',
 ' Echezuria Perozo Jorge E',
 ' Valero Goncalves Yusmary L']